In [1]:
import selenium
from selenium import webdriver

In [2]:
import pandas as pd
from pandas import DataFrame
import time
import re

## Prepping before the scrape
- load the correct site with query search pre-filled
- pre-load number of job posting(starting at 1000 posts)
- define helper functions

In [19]:
# make sure to download the correct chrome driver, update path to its directory
wd = webdriver.Chrome(executable_path='./chromedriver.exe')
url = 'https://www.linkedin.com/jobs/search?keywords=Data%20Analyst&location=San%20Francisco%20Bay%20Area&locationId=&geoId=90000084&sortBy=R&f_TPR=&f_E=2%2C3&position=1&pageNum=0'
wd.get(url)
wd.maximize_window()

In [83]:
# convert str result of job counts into int
no_of_jobs = wd.find_element_by_css_selector('h1>span').get_attribute('innerText')
no_of_jobs_str = no_of_jobs
no_of_jobs_str = no_of_jobs_str[:-1]
no_of_jobs_int = int(no_of_jobs_str.replace(',',''))
no_of_jobs_int

4000

In [84]:
# expanding the listing to get a total of no_of_jobs_in posts or 1000 posts
i = 2

# while i <= int(no_of_jobs_int/25) + 1:
while i <= int(1000/25) + 1: # just want about 1000 postings
    wd.execute_script('window.scrollTo(0,document.body.scrollHeight);')
    i = i+1
    try:
        wd.find_element_by_class_name('infinite-scroller__show-more-button.infinite-scroller__show-more-button').click()
        time.sleep(5)
    except:
        pass
        time.sleep(5)

In [15]:
# filtering out unwanted job posts
def jobIsNotWanted(title):
    if re.search('analyst', title.lower()):
        return False
    return True

In [65]:
# click the show more button to expand the job post description html panel 
def clickShowMore(section):
    buttons = section.find_elements_by_tag_name('button')
    
    if not len(buttons) < 1:
        for button in buttons:
            try:
                if button.text == 'Show more':
                    button.click()
            except:
                pass

## Scraping process:
- defined desire fields(id, title, company, location, description).
- grab all job posting and iterate over each post
- grab and fill the defined fields
- skip over any posting without a populated description panel or missing show more button since they are incomplete data.

In [104]:
# define fields
job_id = []
job_title = []
company_name = []
location = []
description = {}

In [105]:
##### use this block if list expanded to total posting before hand
job_listing = wd.find_element_by_class_name('jobs-search__results-list')
jobs = job_listing.find_elements_by_tag_name('li')

for job in jobs:
    # click on each li ele to populate the job card 
    job.click()
    time.sleep(0.1)
    
    section = wd.find_element_by_xpath('/html/body/div[1]/div/section')
    clickShowMore(section)

#     get job id and title, filtering out existing and non-desirable job based on title
    url = wd.current_url
    url_parts = url.split('JobId=')
    title = job.find_element_by_css_selector('div>h3')
    # move on to next iteration if already exist and job is not wanted
    if((url_parts[1] in job_id or jobIsNotWanted(title.text)) and len(section.text) <= 1): 
        continue
        
    job_id.append(url_parts[1])
    job_title.append(title.text)

    # get the company name
    company = job.find_element_by_css_selector('div>h4')
    company_name.append(company.text)

    # job location
    job_location = job.find_element_by_class_name('job-search-card__location')
    location.append(job_location.text)
    
    # job description
    desc = section.text
    description[url_parts[1]] = desc
    
    time.sleep(1)

In [107]:
print(len(job_id))
print(len(description))

615
615


## Converting data to DF and read/write to .csv files
- save to file so they can be work on at a later time.
- preventing repeating the process if unable to complete. 

In [108]:
job_df = pd.DataFrame()

job_dict = {
    'id': job_id,
    'title': job_title,
    'company_name': company_name,
    'location': location
}

job_df = pd.DataFrame(job_dict)
job_df.to_csv('jobs.csv', index = False)

In [109]:
job_desc_df = pd.DataFrame.from_dict({'job_id': description.keys(), 'description':description.values()})
job_desc_df.to_csv('jobs_desc.csv', index = False)

#### Load files upon restarting job

In [2]:
# load files when restarting job
jobs_df = pd.read_csv('jobs.csv')
jobs_desc_df = pd.read_csv('jobs_desc.csv')

# Scrubbing and handling null values
- find city zipcode from job csv
- remove duplicates, if any, and remove all null for both job/job_desc csv files 

In [28]:
# TODO: 
# add zipcode to job csv 
# remove rows from job_desc and job csv files if description field is null
# sql, python, R, Tableau, power BI, excel, looker, powerpoint, SAS, ETL, Jupyter Notebooks

In [73]:
# drop nulls from desc csv
non_null_jobs_desc = pd.DataFrame(jobs_desc_df.dropna(how = 'any'))
desc_id = non_null_jobs_desc.index
non_null_jobs_desc['desc_id'] = desc_id
non_null_jobs_desc.to_csv('new_jobs_desc.csv', index = False)

486


In [75]:
# dropping rows from job_id csv if they have been drop in desc csv
temp_jobs_df = pd.DataFrame(jobs_df)
temp_job_id = jobs_df['id'].values
job_id_from_desc = non_null_jobs_desc['job_id'].values

for i in temp_job_id:
    if not i in job_id_from_desc:
        row = (temp_jobs_df.loc[temp_jobs_df['id'] == i]).index
        temp_jobs_df.drop(index = row, inplace = True)

temp_jobs_df.to_csv('new_jobs.csv', index = False)

In [29]:
job_df = pd.read_csv('new_jobs.csv')
jobs_desc_df = pd.read_csv('new_jobs_desc.csv')

In [32]:
from uszipcode import SearchEngine, SimpleZipcode, Zipcode

# zipcode finder helper function
def getZipcode(city_name):
    list_zipcode = []
    search = SearchEngine()
    zipcode = search.by_city_and_state(city_name, 'CA')
    if len(zipcode) < 1:
        return None
    return zipcode[0].zipcode

In [36]:
job_locations = job_df['location'].values
location_zipcode = []
for location in job_locations:
    zipc = getZipcode(location)
    location_zipcode.append(zipc)

In [40]:
job_df['zipcode'] = location_zipcode
job_df.to_csv('new_jobs.csv', index = False)

In [9]:
# scrubbing null values for job level
# missing_seniority_level = seniority_level
# missing_seniority_level = [i if len(i) > 0 else 'Not Applicable' for i in seniority_level]

In [10]:
# separate city and state
# state = []
# city = []

# for i in range(len(location)):
#     state.append(location[i][-2:])
#     city.append(location[i][:-4])

In [11]:
# replacing , in company name, just personal ref
# cleaned_company_name = [name.replace(',','-') for name in company_name]